# System Development for Fintech:                                           Anti-Money Laundering Assignment 2023 by Lawrence Pham

### Part 1: Creating the Dataframe

In [1465]:
import re
from typing import Any, Dict, List, Union
import pandas as pd
from datetime import datetime
import pycountry
import os

# Uploading text file
with open(r'C:/Users/Lawrence/Documents/Master Digital Driven Business/System Development for Fintech/MT103 Dataframe.txt', 'r') as g:
    swift = g.read()

# Compiling to identify patterns
patterns = re.compile(r'transaction_\w+:|originator_\w+:|beneficiary_\w+:|instrument_\w+:|incoming_\w+:|outgoing_\w+:')

# Finding matches
matches = patterns.findall(swift)

df = pd.DataFrame(columns=matches)


In [1466]:
# Removing all colons in the column names
df.columns = [col.rstrip(':') for col in df.columns]

In [1467]:
# Viewing the empty dataframe
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,transaction_direction,transaction_status,instrument_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_address,beneficiary_country,beneficiary_account_number,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country


In [1468]:
# Changing datatype of columns where necessary
df = df.astype("object")
df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%d%m%y')
df['transaction_amount'] = pd.to_numeric(df['transaction_amount'], errors='coerce')

In [1469]:
# Viewing changes in datatypes
df.dtypes # ---> transaction amount is now integer, but will become float once we input data 

transaction_date                       datetime64[ns]
transaction_id                                 object
transaction_message                            object
transaction_currency                           object
transaction_amount                              int64
transaction_type                               object
transaction_direction                          object
transaction_status                             object
instrument_type                                object
originator_full_name                           object
originator_first_name                          object
originator_middle_names_patronymic             object
originator_last_name                           object
originator_address                             object
originator_country                             object
originator_account_number                      object
originator_branch_id                           object
originator_bic                                 object
originator_fi_name          

In [1470]:
folder_path = 'C:/Users/Lawrence/Documents/Master Digital Driven Business/System Development for Fintech/SWIFT' 
all_files = os.listdir(folder_path)

## Inputting SWIFT messages in the Dataframe

In [1471]:
# Putting all the parsing code into the for loop so it will append new rows for each SWIFT message
# The creation of the following script was done with the help ChatGPT-3.5 and ChatGPT-4
# An example of the prompt used for this: "Write a python code that searches for field ':52A:' and extracts the string after the '/'"
# After that more specific prompts were used to adjust for differences in SWIFT messages

for file in all_files:
    file_path = os.path.join(folder_path, file)
    
    if os.path.isfile(file_path):
        
        # Read the text file into a string
        with open(file_path, 'r') as f:
            message = f.read()
    
        # Transaction Date
        patt_date = r'(?<=:32A:)\d{6}' # takes the first six characters

        match_date = re.search(patt_date, message)
        if match_date:
            date_str = match_date.group(0)
            trans_date = date_str
        else:
            trans_date = 'None'

        # Transaction ID
        patt_transid = r':20:(.*)\n' # field 20 is the sender's reference
        match = re.search(patt_transid, message)
        if match:
            transaction_id = match.group(1)
            trans_id = transaction_id
        else:
            trans_id = 'None'

        # Transaction Message
        # Regular expression pattern to match :71G: followed by 2 slashes and any text after the second slash
        patt_msg = r":71G:.*\/.*\/(.*)"

        # Search for the pattern in the message string
        match_msg = re.search(patt_msg, message)

        if match_msg:
            # Extract the text after the second slash
            trans_msg = match_msg.group(1)
            trans_msg = re.sub(r"[^a-zA-Z0-9]+$", "", trans_msg)
        else:
            # Search for the pattern :70: followed by any characters until }
            patt_msg_alt = r":70:([^}]*)"
            match_msg_alt = re.search(patt_msg_alt, message)
            
            if match_msg_alt:
                trans_msg = match_msg_alt.group(1).strip()
                # Remove newline characters and merge the lines
                trans_msg = ' '.join(trans_msg.split('\n'))
            else:
                trans_msg = 'None' 

        # Transaction Currency
        patt_curr = r'(?<=:32A:\d{6})[A-Z]{3}' # takes the first three characters after the sixth character in field 32A

        match_curr = re.search(patt_curr, message)
        if match_curr:
            curr_str = match_curr.group(0)
            trans_curr = curr_str
        else:
            trans_curr = 'None'

        # Transaction Amount 
        patt_amt = r'(?<=:32A:\d{6}[A-Z]{3})\d+([.,]\d+)?' # takes all digits after the ninth character in field 32A

        match_amt = re.search(patt_amt, message)
        if match_amt:
            amt_str = match_amt.group(0)
            amt_float = float(amt_str.replace(',', '.'))
            trans_amt = amt_float
        else:
            trans_amt = 'None'


        # Transaction Type
        patt_typ = r'(?<=:23B:)[A-Z]+'  # takes all characters from field 23B

        match_typ = re.search(patt_typ, message)

        if match_typ:
            trans_type = match_typ.group(0)
        else:
            trans_type = 'None'


        # Originator Full name
        patt_ori_full = r":50[AFK]:\s*/[A-Z0-9]+\n((?:\S.*\n)+)"

        # Extracts the text on the first line after :50K:
        match_ori_full = re.search(patt_ori_full, message)
        if match_ori_full:
            # Get the lines of the name
            name_lines = match_ori_full.group(1).split('\n')

            # Check if the first line after :50K: contains at least one space
            if len(name_lines) > 1 and ' ' not in name_lines[0]:
                originator_fullname = name_lines[1]
            else:
                originator_fullname = name_lines[0]


        # Originator First/Middle/Last Names
        if originator_fullname.count(' ') == 1:
            ori_firstname, ori_lastname = originator_fullname.split()
            ori_middlename = 'None'
        elif originator_fullname.count(' ') >= 2:
            names = originator_fullname.split()
            ori_firstname = names[0]
            ori_lastname = names[-1]
            if len(names) > 2:
                ori_middlename = ' '.join(names[1:-1])
            else:
                ori_middlename = 'None'


        # Originator Address
        def extract_lines_after_50k(message, lines_to_extract):
            message_lines = message.split('\n')
            result = []

            pattern = re.compile(r'^:(50K|50F|50A):')

            for index, line in enumerate(message_lines):
                if pattern.match(line):
                    for n in lines_to_extract:
                        result.append(message_lines[index + n])

                    return '\n'.join(result)

        lines_to_extract = [2, 3]
        result_lines = extract_lines_after_50k(message, lines_to_extract)

        def contains_numbers(s):
            return bool(re.search(r'\d', s))

        result_lines_list = result_lines.split('\n')
        ori_add = ''

        for i, s in enumerate(result_lines_list):
            if contains_numbers(s):
                ori_add = result_lines_list[i-1] + ' ' + s
                break
            else:
                ori_add = 'None'


        # Originator Account Number
        patt_ori_actnr = r'(?<=\n):50[AFK]:(.+)'
        matches_actnr = re.findall(patt_ori_actnr, message)

            # Extract account number after :50[AFK]: and removing '/'
        for match_actnr in matches_actnr:
            match_actnr = match_actnr.strip()
            if match_actnr.startswith('/'):
                match_actnr = match_actnr[1:]
                ori_actnr = match_actnr
            else:
                ori_actnr = 'None'

        # Originator Country 

        # Originator Bank Identifier Code
        patt_ori_bic = r'(?<=\n):52[ABD]:(.+)'
        patt_alt_ori_bic = r'(?<=\n):50[ABD]:.*\n(.+)'

        match_bic = re.search(patt_ori_bic, message)
        if match_bic:
            ori_bic = match_bic.group(1)
            ori_bic = ori_bic.strip().replace("/", "")
        else:
            match_alt_bic = re.search(patt_alt_ori_bic, message)
            if match_alt_bic:
                ori_bic = match_alt_bic.group(1)
                ori_bic = ori_bic.strip().replace("/", "")
                if len(ori_bic) == 8:
                    ori_bic += 'XXX'
            else:
                ori_bic = 'None'
        
        # Originator FI Name
        patterns = [r'(?<=\n):52[ABD]:.*\n(.*)']
        ori_fi_name = None

        for pattern in patterns:
            match = re.search(pattern, message)
            if match:
                ori_fi_name = match.group(1).strip()
                break
            else:
                ori_fi_name = 'None'


        # Receiving Intermediary Financial Institution BIC
        patt_rec_bic = r'(?<=\n):56[ABD]:(.+)' # Field 56 is designated for the intermediary bank

        match_rec_bic = re.search(patt_rec_bic, message)
        if match_rec_bic:
            rec_bic = match_rec_bic.group(1)
            rec_bic = rec_bic.strip().replace("/", "")
        else:
            rec_bic = 'None'

        # Sending Intermediary Financial Institution BIC
        patt_send_bic = r'(?<=\n):57[ABD]:(.+)' # Field 57 is the account with institution

        match_send_bic = re.search(patt_send_bic, message)
        if match_send_bic:
            send_bic = match_send_bic.group(1)
            send_bic = send_bic.strip().replace("/", "")
        else:
            send_bic = 'None'


        # Beneficiary full name
        def extract_line_after_59(message):
            message_lines = message.split('\n')
            for index, line in enumerate(message_lines):
                if line.startswith(':59:'):
                    next_line = message_lines[index + 1]
                    if 'unset' in next_line.lower():
                        return message_lines[index + 2]
                    else:
                        return next_line

        ben_fullname = extract_line_after_59(message)

        # Beneficiary First/Middle/Last Names
        if ben_fullname.count(' ') == 1:
            ben_firstname, ben_lastname = ben_fullname.split()
            ben_middlename = 'None'
        elif ben_fullname.count(' ') >= 2:
            names = ben_fullname.split()
            ben_firstname = names[0]
            ben_lastname = names[-1]
            if len(names) > 2:
                ben_middlename = ' '.join(names[1:-1])
            else:
                ben_middlename = 'None'

        # Beneficiary Address
        def extract_lines_after_59(message, lines_to_extract):
            message_lines = message.split('\n')
            result = []

            pattern = re.compile(r'^:59:')

            for index, line in enumerate(message_lines):
                if pattern.match(line):
                    for n in lines_to_extract:
                        result.append(message_lines[index + n])

                    return '\n'.join(result)

        lines_to_extract = [1, 2, 3, 4, 5]
        result_lines = extract_lines_after_59(message, lines_to_extract)

        def contains_numbers(s):
            return bool(re.search(r'\d', s))

        result_lines_list = result_lines.split('\n')
        ben_add = ''

        for i, s in enumerate(result_lines_list):
            if contains_numbers(s) and re.search(r'[a-zA-Z]', s) and not s.startswith(':'):
                ben_add += s + ', '

        if ben_add:
            ben_add = ben_add.rstrip(', ')
        else:
            ben_add = 'None'


        # Beneficiary Country

        # Regular expression pattern to extract the line above either :70: or :71A:
        patt_text = r"(.*?)(?=\n(:70:|:71[AFG]:))"

        # Search for the line above :70: or :71A: in the message
        matches = re.search(patt_text, message, re.MULTILINE)
        if matches:
            # Get the matched line_above
            line_above = matches.group(1).strip()

            # Extract the country from the line_above
            words = line_above.split()
            for word in words:
                try:
                    ben_cnty = pycountry.countries.lookup(word)
                    ben_cnty_iso = ben_cnty.alpha_2
                    break
                except LookupError:
                    continue


        # Beneficiary Account Number
        patt_ben_actnr = r'(?<=\n):59[AF]?:(.+)'
        matches_ben_actnr = re.findall(patt_ben_actnr, message)

        def count_digits(string):
            return sum(c.isdigit() for c in string)

        # Extract beneficiary account number after :59[AF]: and removing '/'
        for match_ben_actnr in matches_ben_actnr:
            match_ben_actnr = match_ben_actnr.strip()
            if match_ben_actnr.startswith('/'):
                match_ben_actnr = match_ben_actnr[1:]
                if count_digits(match_ben_actnr) >= 4:
                    ben_actnr = match_ben_actnr
                else:
                    ben_actnr = 'None'
            else:
                ben_actnr = 'None'

        # Beneficiary Bank Identifier Code
        patt_ben_bic = r'(?<=\n):57[ABCD]:(.+)'

        matches_ben_bic = re.findall(patt_ben_bic, message)

            # Extract account number after :50[AFK]: and removing '/'
        for match_ben_bic in matches_ben_bic:
            match_ben_bic = match_ben_bic.strip()
            if match_ben_bic.startswith('/'):
                match_ben_bic = match_ben_bic[1:]
                ben_bic = match_ben_bic
                if len(ben_bic) == 8:
                    ben_bic += 'XXX'
            else:
                ben_bic = 'None'

        # Beneficiary Financial Institution Name
        patt_ben_fi = r":57[ABCD]:\s*\/.*\n(.*)"

            # Extracts the text on the second line after :57:
        match_ben_fi = re.search(patt_ben_fi, message)
        if match_ben_fi:
            ben_fi = match_ben_fi.group(1)
        else:
            ben_fi = 'None'

        # Beneficiary Financial Institution Country
        patt_ben_fi_cnty = r":57[ABCD]:\s*\/.*\n.*\n.*, (.*)"

            # Extract the text after the comma in the third line after :50K:
        match_ben_fi_cnty = re.search(patt_ben_fi_cnty, message)
        if match_ben_fi_cnty:
            ben_fi_cnty = match_ben_fi_cnty.group(1)
        else:
            ben_fi_cnty = 'None'

        data = {
            'transaction_date': trans_date,
            'transaction_id': trans_id,
            'transaction_message': trans_msg,
            'transaction_currency': trans_curr,
            'transaction_amount': trans_amt,
            'transaction_type': trans_type,
            'originator_full_name': originator_fullname,
            'originator_first_name': ori_firstname,
            'originator_middle_names_patronymic': ori_middlename,
            'originator_last_name': ori_lastname,
            'originator_address': ori_add,
            'originator_country': ori_actnr[:2],
            'originator_account_number': ori_actnr,
            'originator_branch_id': ori_bic[-3:],
            'originator_bic': ori_bic,
            'originator_fi_name': ori_fi_name,
            'originator_fi_country': ori_actnr[:2],
            'incoming_intermediary_fi_bic': rec_bic,
            'outgoing_intermediary_fi_bic': send_bic,
            'beneficiary_full_name': ben_fullname,
            'beneficiary_first_name': ben_firstname,
            'beneficiary_middle_names_patronymic': ben_middlename,
            'beneficiary_last_name': ben_lastname,
            'beneficiary_address': ben_add,
            'beneficiary_country': ben_cnty_iso,
            'beneficiary_account_number': ben_actnr,
            'beneficiary_branch_id': ben_bic[-3:],
            'beneficiary_bic': ben_bic,
            'beneficiary_fi_name': ben_fi,
            'beneficiary_fi_country': ben_bic[4:6]
            }

        df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)

In [1472]:
df = df.drop('transaction_direction', axis=1).drop('transaction_status', axis=1).drop('instrument_type', axis=1) # dropping unused columns

In [1473]:
pd.set_option('display.max_columns', None) # showing all 33 columns
df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%d%m%y') # again, because it was not adjusted properly in the beginning
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_address,beneficiary_country,beneficiary_account_number,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country
0,2022-03-21,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000.0,CRED,COMMERZBANK AG,COMMERZBANK,None,AG,None,DE,DE98765432101234567890,XXX,COBADEHHXXX,COMMERZBANK AG,DE,None,HBUKGB4BXXX,NORDFISCH GMBH,NORDFISCH,None,GMBH,"BODENSEE STR. 226, 22761 HAMBURG",DE,GB57METR12345678901234,XXX,HBUKGB4BXXX,HSBC BANK PLC,GB
1,2022-03-21,MT103 0001,PAYMENT FOR GOODS,USD,10000.0,CRED,ABC INDUSTRIES,ABC,None,INDUSTRIES,"NEW YORK, NY 10001 123 MAIN STREET",US,US12345678901234567890,XXX,ABCBUS33XXX,ABC BANK,US,ICBKCNBJGZU,CITIUS33,XYZ SUPPLIERS,XYZ,None,SUPPLIERS,123 HUANGPU ROAD,CN,CN123456789012345678,XXX,CITIUS33XXX,CITIBANK NA,US
2,2022-03-21,MT103 0001,INV NO. 12345 REF. 98765 SUPPLY OF GOODS AS PE...,USD,9899.0,CRED,ABC SUPPLIERS BV,ABC,SUPPLIERS,BV,None,NL,NL20ABNA0404875234,XXX,ABNANL2AXXX,None,NL,SCBLGB2LXXX,BNYMUS33XXX,AFRICAN EXPORT-IMPORT BANK,AFRICAN,EXPORT-IMPORT,BANK,None,NG,None,XXX,BNYMUS33XXX,BNY MELLON,US


## Part 2: Checking for Money Laundering

### Rounding Fraud

"Fraudsters tend to favor round numbers even in situations where round numbers should be rare or nonexistent (M. Nigrini, 2020)."

Transactions with round numbers in itself are not a direct indication of a fraudulent transaction, but in combination with other common money laundering indicators it could increase the probablity of a fraudulent transaction.

Nigrini, M. J., PhD. (2020, December 1). Fraud Magazine. https://www.fraud-magazine.com/cover-article.aspx?id=4295011516

In [1474]:
# Flagging transaction with round numbers at an interval of 100
# Prompt used for ChatGPT-4: "Write a Python code that searches through the dataframe and checks if the transaction amounts in the 'transaction_amount' column are 
# round numbers and divisible by 100. Add a column 'rounding', if round then 1, else 0."

for index, row in df.iterrows():
    trans_amt = row['transaction_amount']
    if trans_amt % 100 == 0:
        df.loc[index, 'rounding'] = 1
        print(f"Transaction {index + 1}: {trans_amt} is a round number.")
    else:
        df.loc[index, 'rounding'] = 0
        print(f"Transaction {index + 1}: {trans_amt} is not a round number.")

df['rounding'] = df['rounding'].astype(int)

Transaction 1: 5000.0 is a round number.
Transaction 2: 10000.0 is a round number.
Transaction 3: 9899.0 is not a round number.


### High Risk Countries & Tax Haven Transactions

High Risk Countries are countries and jurisdictions that have been identified by the FATF to have weak measures to combat money laundering and terrorist financing (FATF, 2023). The FATF is a intergovernmental organisation that was established in 1989 by the G7 countries. As of 2022, the FATF is comprised of 37 member jurisdictions and 2 regional organisations, representing most financial centres around the world (FATF, 2023). As stated on their official website: "The objectives of the FATF are to set standards and promote effective implementation of legal, regulatory and operational measures for combating money laundering, terrorist financing and other related threats to the integrity of the international financial system (FATF, 2023)."

Compared to the list of high-risk countries, the lists of tax haven countries are generally considered more subjective. This is because while the integration of illegally obtained financial assets into the legal financial system, known as money laundering, is a crime, the use of creative accounting structures to minimize tax liability is not illegal, which means that the criteria for identifying tax havens can vary depending on the specific jurisdiction or international standards. For example, the EU has comprised a list of tax haven countries which does not include its own members. This may lead to a conflict in interest or bias when creating a code that flags transactions. For that reason, we have examined multiple lists from different governmental and non-governmental organisations to compose a list of tax haven countries (Unger et al., 2020).

The final list is comprised of the top 16 countries from the Tax Justice Network. These countries have been given a Corporate Tax Haven Index (CTHI), and shows the share for the world's corporate tax abuse per country. The total lists contains 70 countries, but we chose to only take the top 16 since these countries together are responsible for approximately 66% of the world's corporate tax abuse (Tax Justice Network, n.d.).

“Black and grey” lists. (2023, February). Financial Action Task Force (FATF). Retrieved March 31, 2023, from https://www.fatf-gafi.org/en/countries/black-and-grey-lists.html

Members and Observers. (2023, March 29). Financial Action Task Force (FATF). Retrieved March 31, 2023, from https://web.archive.org/web/20211124012308/https://www.fatf-gafi.org/about/membersandobservers/

Unger, B., Alshut, F., Jeroschheroldda Costa Reis, J., Blokland, G., & Rosell Flores, L. (2020, May). Improving Anti-Money Laundering Policy. European Parliament. Retrieved March 31, 2023, from https://www.europarl.europa.eu/RegData/etudes/STUD/2020/648789/IPOL_STU(2020)648789_EN.pdf

What we do. (n.d.). Financial Action Task Force (FATF). Retrieved March 31, 2023, from https://www.fatf-gafi.org/en/the-fatf/what-we-do.html

Tax Justice Network. (n.d.). Corporate Tax Haven Index - 2021 Results. Retrieved March 31, 2023, from https://cthi.taxjustice.net/en/cthi/cthi-2021-results

In [1475]:
# The lists of countries have been placed in a .txt file in the following format: "Netherlands: NL (ISO Alpha-2 code), NLD (ISO Alpha-3 code)""

# The blacklist is established by the Financial Action Task Force  (https://www.fatf-gafi.org/en/countries/black-and-grey-lists.html) 
with open(r'C:/Users/Lawrence/Documents/Master Digital Driven Business/System Development for Fintech/Black List Countries.txt', 'r') as x:
    blacklist = x.read()

# The greylist is established by the Financial Action Task Force  (https://www.fatf-gafi.org/en/countries/black-and-grey-lists.html) 
with open(r'C:/Users/Lawrence/Documents/Master Digital Driven Business/System Development for Fintech/Grey List Countries.txt', 'r') as y:
    greylist = y.read()

# The corporate tax haven index list is established by the Tax Justice Network (https://cthi.taxjustice.net/en/cthi/cthi-2021-results)
with open(r'C:/Users/Lawrence/Documents/Master Digital Driven Business/System Development for Fintech/CTHI Countries.txt', 'r') as z:
    cthilist = z.read()

In [1476]:
# Split the contents of the file by each new line
linesx = blacklist.split('\n')
linesy = greylist.split('\n')
linesz = cthilist.split('\n')

In [1477]:
black_list = []

for linex in linesx:
    blacklistx = linex.split(': ')[1].split(' (ISO Alpha-2 code)')[0]
    black_list.append(blacklistx)

print(black_list)

['KP', 'IR', 'MM']


In [1478]:
grey_list = []

for liney in linesy:
    greylisty = liney.split(': ')[1].split(' (ISO Alpha-2 code)')[0]
    grey_list.append(greylisty)

print(grey_list)

['AL', 'BB', 'BF', 'KY', 'CD', 'GI', 'HT', 'JM', 'JO', 'ML', 'MZ', 'NG', 'PA', 'PH', 'SN', 'ZA', 'SS', 'SY', 'TZ', 'TR', 'UG', 'AE', 'YE']


In [1479]:
high_risk = black_list + grey_list

print(high_risk)

['KP', 'IR', 'MM', 'AL', 'BB', 'BF', 'KY', 'CD', 'GI', 'HT', 'JM', 'JO', 'ML', 'MZ', 'NG', 'PA', 'PH', 'SN', 'ZA', 'SS', 'SY', 'TZ', 'TR', 'UG', 'AE', 'YE']


In [1480]:
cthi_list = []

for linez in linesz:
    cthilistz = linez.split(': ')[1].split(' (ISO Alpha-2 code)')[0]
    cthi_list.append(cthilistz)

print(cthi_list)

['VG', 'KY', 'BM', 'NL', 'CH', 'LU', 'HK', 'JE', 'SG', 'AE', 'IE', 'BS', 'GB', 'CY', 'MU', 'BE']


In [1481]:
# Creating a function to flag payments from high risk countries to tax haven countries
# Prompt used for ChatGPT-4: "Write a Python script that creates a function to identify and flag payments based on their originator and beneficiary countries. 
# The function should flag payments from high-risk countries to tax haven countries with a risk level of 2, 
# while payments from or to either high-risk countries or tax haven countries should be flagged with a risk level of 1. 
# Apply the function to a DataFrame and create a new column called 'risky' with the results."
def should_flag_payment(row):
    if row['originator_country'] in high_risk and row['beneficiary_country'] in cthi_list:
        return 2
    elif row['beneficiary_country'] in high_risk:
        return 1
    elif row['beneficiary_country'] in cthi_list:
        return 1
    else:
        return 0

# Apply the should_flag_payment function to the DataFrame and create a new column with the result
df['risky'] = df.apply(should_flag_payment, axis=1)

# Filter the DataFrame to show only flagged payments
flagged_payments = df[df['risky'] == 1]

In [1482]:
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_address,beneficiary_country,beneficiary_account_number,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,rounding,risky
0,2022-03-21,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000.0,CRED,COMMERZBANK AG,COMMERZBANK,None,AG,None,DE,DE98765432101234567890,XXX,COBADEHHXXX,COMMERZBANK AG,DE,None,HBUKGB4BXXX,NORDFISCH GMBH,NORDFISCH,None,GMBH,"BODENSEE STR. 226, 22761 HAMBURG",DE,GB57METR12345678901234,XXX,HBUKGB4BXXX,HSBC BANK PLC,GB,1,0
1,2022-03-21,MT103 0001,PAYMENT FOR GOODS,USD,10000.0,CRED,ABC INDUSTRIES,ABC,None,INDUSTRIES,"NEW YORK, NY 10001 123 MAIN STREET",US,US12345678901234567890,XXX,ABCBUS33XXX,ABC BANK,US,ICBKCNBJGZU,CITIUS33,XYZ SUPPLIERS,XYZ,None,SUPPLIERS,123 HUANGPU ROAD,CN,CN123456789012345678,XXX,CITIUS33XXX,CITIBANK NA,US,1,0
2,2022-03-21,MT103 0001,INV NO. 12345 REF. 98765 SUPPLY OF GOODS AS PE...,USD,9899.0,CRED,ABC SUPPLIERS BV,ABC,SUPPLIERS,BV,None,NL,NL20ABNA0404875234,XXX,ABNANL2AXXX,None,NL,SCBLGB2LXXX,BNYMUS33XXX,AFRICAN EXPORT-IMPORT BANK,AFRICAN,EXPORT-IMPORT,BANK,None,NG,None,XXX,BNYMUS33XXX,BNY MELLON,US,0,1


### Smurfing

"Smurfing is a money-laundering technique involving the structuring of large amounts of cash into multiple small transactions (Investopedia, 2023)."

For example, someone who has acquired EUR 10.000 cash from selling illegal contraband might enlist 10 people who will each deposit EUR 1.000 into their own account, and then transfer it into the criminal's private account or business account.

The following code is a simplified transaction monitoring technique that will give a warning if an account has more than 5 transaction of EUR>500 & EUR<10.000 to the same beneficiary account within a month. Due to insufficient transaction examples, the code will not give an output.

Besides that, it is also possible to do the same for a (private) benefiary account if it receives more than 10 transaction from different bank accounts with amounts that end with four zeroes, within a month for example.

However, writing such a code for business accounts may be more difficult. It should then probably take into account the industry and perhaps the average income per month in a certain region, or the income history of that company. There are many variables at involved, thus writing such code requires extensive knowledge on forensic accounting and machine learning for AML.

Detecting smurfing is more nuanced than the examples above, since it should detect unusual transaction behaviour. The parameters could be relative depending on a person's income and wealth. So, machine learning algorithms should be trained to identify certain transaction patterns and trigger a warning when anomalies occur. 

Hayes, A. (2021, July 1). What Is a Smurf and How Does Smurfing Work? Investopedia. https://www.investopedia.com/terms/s/smurf.asp#:~:text=Smurfing%20is%20a%20money%2Dlaundering,reporting%20limits%20and%20avoid%20detection.

In [1483]:
# Filter transactions with an amount greater than 500 and less than 10.000
# Reason for 500> X <10.000 is because it is unlikely that someone wires "large" sums of money to the same account within a month.
# It should be noted that this flag is not conclusive because the amounts are relative, spending behaviour varies per person. 
# Other suspicious behaviours within the accounts should be assessed before taking action.

filtered_transactions = df[(df['transaction_amount'] > 500) & (df['transaction_amount'] < 10000)]

# Group transactions by originator, beneficiary, and month, then count transactions
grouped_transactions = filtered_transactions.groupby([
    'originator_account_number',
    'beneficiary_account_number',
    pd.Grouper(key='transaction_date', freq='M')
]).size().reset_index(name='transaction_count')

# Filter rows with more than 5 transactions in a month
flagged_transactions = grouped_transactions[grouped_transactions['transaction_count'] > 5]

def is_smurfing(row, flagged_transactions):
    if row['originator_account_number'] in flagged_transactions['originator_account_number'].values and \
       row['beneficiary_account_number'] in flagged_transactions['beneficiary_account_number'].values and \
       row['transaction_date'].to_period('M') in flagged_transactions['transaction_date'].values:
        return 1
    else:
        return 0

df['smurf'] = df.apply(lambda row: is_smurfing(row, flagged_transactions), axis=1)

# Display flagged transactions
print(flagged_transactions)

Empty DataFrame
Columns: [originator_account_number, beneficiary_account_number, transaction_date, transaction_count]
Index: []


In [1484]:
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_address,beneficiary_country,beneficiary_account_number,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,rounding,risky,smurf
0,2022-03-21,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000.0,CRED,COMMERZBANK AG,COMMERZBANK,None,AG,None,DE,DE98765432101234567890,XXX,COBADEHHXXX,COMMERZBANK AG,DE,None,HBUKGB4BXXX,NORDFISCH GMBH,NORDFISCH,None,GMBH,"BODENSEE STR. 226, 22761 HAMBURG",DE,GB57METR12345678901234,XXX,HBUKGB4BXXX,HSBC BANK PLC,GB,1,0,0
1,2022-03-21,MT103 0001,PAYMENT FOR GOODS,USD,10000.0,CRED,ABC INDUSTRIES,ABC,None,INDUSTRIES,"NEW YORK, NY 10001 123 MAIN STREET",US,US12345678901234567890,XXX,ABCBUS33XXX,ABC BANK,US,ICBKCNBJGZU,CITIUS33,XYZ SUPPLIERS,XYZ,None,SUPPLIERS,123 HUANGPU ROAD,CN,CN123456789012345678,XXX,CITIUS33XXX,CITIBANK NA,US,1,0,0
2,2022-03-21,MT103 0001,INV NO. 12345 REF. 98765 SUPPLY OF GOODS AS PE...,USD,9899.0,CRED,ABC SUPPLIERS BV,ABC,SUPPLIERS,BV,None,NL,NL20ABNA0404875234,XXX,ABNANL2AXXX,None,NL,SCBLGB2LXXX,BNYMUS33XXX,AFRICAN EXPORT-IMPORT BANK,AFRICAN,EXPORT-IMPORT,BANK,None,NG,None,XXX,BNYMUS33XXX,BNY MELLON,US,0,1,0


## Nesting

"Downstream correspondent clearance (or nesting) refers to the use of a bank's correspondent relationship by a number of underlying banks or financial institutions through their relationships with the correspondent bank’s direct customer (AUSTRAC, n.d.)."


AUSTRAC. (n.d.). Due diligence of correspondent banking relationships. Australian Government. Retrieved April 1, 2023, from https://www.austrac.gov.au/due-diligence-correspondent-banking-relationships#:~:text=Downstream%20correspondent%20clearance%20(or%20nesting,the%20correspondent%20bank's%20direct%20customer.

## Non Adherence to FATF Recommendation 16 

### Wire transfers
"Countries should ensure that financial institutions include required and accurate originator information, and required beneficiary information, on wire transfers and related messages, and that the information remains with the wire transfer or related message throughout the paymentchain (FATF, 2023)." 

"Countries should ensure that financial institutions monitor wire transfers for the purpose of detecting those which lack required originator and/or beneficiary information, and take appropriate measures (FATF, 2023)."

"Countries should ensure that, in the context of processing wire transfers, financial institutions take freezing action and should prohibit conducting transactions with designated persons and entities, as per the obligations set out in the relevant United Nations Security Council resolutions, such as resolution 1267 (1999) and its successor resolutions, and resolution 1373(2001), relating to the prevention and suppression of terrorism and terrorist financing (FATF, 2023)."

In other words, all transactions must contain originator's and beneficiary's full name; address; and account number. 

FATF (2012-2023), International Standards on Combating Money Laundering and the Financing of 
Terrorism & Proliferation, FATF, Paris, France, https://www.fatf-gafi.org/content/dam/recommandations/FATF%20Recommendations%202012.pdf.coredownload.inline.pdf

In [1485]:
# Prompt used for ChatGPT-4: "Write a Python script that checks if specific columns in a DataFrame have non-empty and non-null values.
# The columns to check are 'originator_full_name', 'originator_address', 'originator_account_number', 'beneficiary_full_name', 'beneficiary_address', and
# 'beneficiary_account_number'. If all these columns have non-empty and non-null values, return '0'; otherwise, return '1'. 
# Apply this check to a DataFrame and create a new column called 'FATF' with the results."

def update_transaction_status(row):
    required_columns = [
        'originator_full_name',
        'originator_address',
        'originator_account_number',
        'beneficiary_full_name',
        'beneficiary_address',
        'beneficiary_account_number',
    ]
    if all(pd.notna(row[column]) and row[column].strip() != '' and row[column] != 'None' for column in required_columns):
        return '0'
    else:
        return '1'

# Assuming df is your DataFrame
df['FATF'] = df.apply(update_transaction_status, axis=1)

In [1486]:
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_address,beneficiary_country,beneficiary_account_number,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,rounding,risky,smurf,FATF
0,2022-03-21,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000.0,CRED,COMMERZBANK AG,COMMERZBANK,None,AG,None,DE,DE98765432101234567890,XXX,COBADEHHXXX,COMMERZBANK AG,DE,None,HBUKGB4BXXX,NORDFISCH GMBH,NORDFISCH,None,GMBH,"BODENSEE STR. 226, 22761 HAMBURG",DE,GB57METR12345678901234,XXX,HBUKGB4BXXX,HSBC BANK PLC,GB,1,0,0,1
1,2022-03-21,MT103 0001,PAYMENT FOR GOODS,USD,10000.0,CRED,ABC INDUSTRIES,ABC,None,INDUSTRIES,"NEW YORK, NY 10001 123 MAIN STREET",US,US12345678901234567890,XXX,ABCBUS33XXX,ABC BANK,US,ICBKCNBJGZU,CITIUS33,XYZ SUPPLIERS,XYZ,None,SUPPLIERS,123 HUANGPU ROAD,CN,CN123456789012345678,XXX,CITIUS33XXX,CITIBANK NA,US,1,0,0,0
2,2022-03-21,MT103 0001,INV NO. 12345 REF. 98765 SUPPLY OF GOODS AS PE...,USD,9899.0,CRED,ABC SUPPLIERS BV,ABC,SUPPLIERS,BV,None,NL,NL20ABNA0404875234,XXX,ABNANL2AXXX,None,NL,SCBLGB2LXXX,BNYMUS33XXX,AFRICAN EXPORT-IMPORT BANK,AFRICAN,EXPORT-IMPORT,BANK,None,NG,None,XXX,BNYMUS33XXX,BNY MELLON,US,0,1,0,1


## Shell Company Characteristics

"A shell corporation is a corporation without active business operations or significant assets. These types of corporations are not all necessarily illegal, but they are sometimes used illegitimately, such as to disguise business ownership from law enforcement or the public (Kenton, 2022)."

Since shell companies serve no other purpose than disguising the ultimate beneficial owner or avoiding/limiting taxes, it might be a reason that people do not any passion or thought behind naming the company. However, more research is required to confirm this proposition. Nevertheless, the code below checks for company names that have consecutive numbers or letters in it, and that are located in tax haven countries from the cthi_list.

Kenton, W. (2022, July 17). What Is a Shell Corporation? How It’s Used, Examples and Legality. Investopedia. Retrieved March 31, 2023, from https://www.investopedia.com/terms/s/shellcorporation.asp#:~:text=A%20shell%20corporation%20is%20a,law%20enforcement%20or%20the%20public.

In [1487]:
# Prompt used for ChatGPT-4: "Create a Python function called 'check_consecutive_letters_or_digits' that takes a name as input and returns 
# True if it contains consecutive alphabetic letters or digits in sequences like 'abc', '345', etc. 
# Then, apply this function to both the 'originator_full_name' and 'beneficiary_full_name' columns in a DataFrame. 
# If either of these columns contains consecutive sequences, flag the row by assigning a value of '1' in a new column named 'shell'.
# And if the 'beneficiary_country' is in a list of tax haven countries, it will assign a value of '2. 
# Filter the DataFrame to display only flagged rows and print the result."

def check_consecutive_letters_or_digits(name):
    return 1 if re.search(r'(abc|bcd|cde|def|efg|fgh|ghi|hij|ijk|jkl|klm|lmn|mno|nop|opq|pqr|qrs|rst|stu|tuv|uvw|vwx|wxy|xyz|012|123|234|345|456|567|678|789)', 
    name.lower()) else 0

# Create a new column 'shell' to flag rows based on the conditions
df['shell'] = df.apply(lambda row: check_consecutive_letters_or_digits(row['originator_full_name']) +
                       check_consecutive_letters_or_digits(row['beneficiary_full_name']) +
                       (row['beneficiary_country'] in cthi_list) * 1, axis=1)

# Filter the DataFrame to show only shell rows with value 1 or 2
flagged_rows = df[df['shell'].isin([1, 2])]

# Print the shell rows
print(flagged_rows)

  transaction_date transaction_id  \
1       2022-03-21     MT103 0001   
2       2022-03-21     MT103 0001   

                                 transaction_message transaction_currency  \
1                                  PAYMENT FOR GOODS                  USD   
2  INV NO. 12345 REF. 98765 SUPPLY OF GOODS AS PE...                  USD   

   transaction_amount transaction_type originator_full_name  \
1             10000.0             CRED       ABC INDUSTRIES   
2              9899.0             CRED     ABC SUPPLIERS BV   

  originator_first_name originator_middle_names_patronymic  \
1                   ABC                               None   
2                   ABC                          SUPPLIERS   

  originator_last_name                  originator_address originator_country  \
1           INDUSTRIES  NEW YORK, NY 10001 123 MAIN STREET                 US   
2                   BV                                None                 NL   

  originator_account_number origina

In [1488]:
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_address,beneficiary_country,beneficiary_account_number,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,rounding,risky,smurf,FATF,shell
0,2022-03-21,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000.0,CRED,COMMERZBANK AG,COMMERZBANK,None,AG,None,DE,DE98765432101234567890,XXX,COBADEHHXXX,COMMERZBANK AG,DE,None,HBUKGB4BXXX,NORDFISCH GMBH,NORDFISCH,None,GMBH,"BODENSEE STR. 226, 22761 HAMBURG",DE,GB57METR12345678901234,XXX,HBUKGB4BXXX,HSBC BANK PLC,GB,1,0,0,1,0
1,2022-03-21,MT103 0001,PAYMENT FOR GOODS,USD,10000.0,CRED,ABC INDUSTRIES,ABC,None,INDUSTRIES,"NEW YORK, NY 10001 123 MAIN STREET",US,US12345678901234567890,XXX,ABCBUS33XXX,ABC BANK,US,ICBKCNBJGZU,CITIUS33,XYZ SUPPLIERS,XYZ,None,SUPPLIERS,123 HUANGPU ROAD,CN,CN123456789012345678,XXX,CITIUS33XXX,CITIBANK NA,US,1,0,0,0,2
2,2022-03-21,MT103 0001,INV NO. 12345 REF. 98765 SUPPLY OF GOODS AS PE...,USD,9899.0,CRED,ABC SUPPLIERS BV,ABC,SUPPLIERS,BV,None,NL,NL20ABNA0404875234,XXX,ABNANL2AXXX,None,NL,SCBLGB2LXXX,BNYMUS33XXX,AFRICAN EXPORT-IMPORT BANK,AFRICAN,EXPORT-IMPORT,BANK,None,NG,None,XXX,BNYMUS33XXX,BNY MELLON,US,0,1,0,1,1


## Trade Based Money Laundering

"Trade-based money laundering (TBML) is defined as the process of disguising the proceeds of crime and moving value through the use of trade transactions in an attempt to legitimise their illicit origins. In practice, this can be achieved through the misrepresentation of the price, quantity or quality of imports or exports. Moreover, trade-based money laundering techniques vary in complexity and are frequently used in combination with other money laundering techniques to further obscure the money trail (FATF, n.d.)."

According to the FATF (2006), there are four basic TBML techniques; (1) Over- and underinvoicing of goods and services; (2) Multiple-invoicing of goods and services; (3) Over- and under-shipment of goods and services; (4) Falsely describing goods and services. 

Effectively tackling these basic TBML techniques may be a challenge with the limited SWIFT examples provided for this assignment. 

Nevertheless, multiple invoicing could potentially be tracked by monitoring invoice numbers and/or reference numbers of transactions to spot anomalies, for instance if the invoice number is used more than twice in the same year by the same originator account number. Or if the invoice number is looking suspicious, i.e., consecutive numbering. 

Secondly, a method to track falsely describing goods and services and/or over- and underinvoicing, may be to look at the overall industry that the money laundering company is alledgly operating in. For example, SWIFT msg (1) states: "THIS IS A PAYMENT FOR TUNA SUPPLY", then we can analyze transactions from other tuna fish import companies and see how common the transaction amount and distination is. However, this probably requires extensive data of the industry and annual transactions.

Moreover, over- and under-invoicing/ shipment of goods and services may also require regular auditing of inventory in order to determine whether it is actual or TBML.

So, for this assignment we could write a script that analyzes invoice and reference number that correspond to originator and beneficiary accounts, and flag them if they occur more than twice within a certain timeframe, or if the numbers are 'suspicious' (111, 123, 321, etc.).

Financial Action Task Force (FATF). (n.d.). Trade-Based Money Laundering. Retrieved March 31, 2023, from https://www.fatf-gafi.org/en/publications/Methodsandtrends/Trade-basedmoneylaundering.html

Financial Action Task Force (FATF). (2006). Trade Based Money Laundering. FATF/OECD. https://www.fatf-gafi.org/en/publications/Methodsandtrends/Trade-basedmoneylaundering.html

In [1489]:
# Prompt used for ChatGPT-4: "Write a script that analyzes invoice and reference number that correspond to originator and beneficiary accounts, 
# and flag them if they occur more than twice within a certain timeframe, or if the numbers are 'suspicious' (111, 123, 321, etc.)."

# Create a helper function to check if a number is suspicious
def is_suspicious_number(number):
    suspicious_numbers = [
        '111', '222', '333', '444', '555', '666', '777', '888', '999',
        '123', '234', '345', '456', '567', '678', '789', '987', '876',
        '765', '654', '543', '432', '321'
    ]
    for suspicious in suspicious_numbers:
        if suspicious in number:
            return True
    return False


# Create a function to flag suspicious transactions
def flag_suspicious_transactions(row):
    invoice_suspicious = is_suspicious_number(row['transaction_message'])
    
    if invoice_suspicious:
        return 1
    else:
        return 0

# Apply the flag_suspicious_transactions function to the DataFrame and create a new column 'suspicious'
df['TBML'] = df.apply(flag_suspicious_transactions, axis=1)

In [1490]:
# Any transaction that contains a 'suspicious' string of numbers in the transaction message will be marked with '1' in the suspicious column.
df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_address,beneficiary_country,beneficiary_account_number,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,rounding,risky,smurf,FATF,shell,TBML
0,2022-03-21,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000.0,CRED,COMMERZBANK AG,COMMERZBANK,None,AG,None,DE,DE98765432101234567890,XXX,COBADEHHXXX,COMMERZBANK AG,DE,None,HBUKGB4BXXX,NORDFISCH GMBH,NORDFISCH,None,GMBH,"BODENSEE STR. 226, 22761 HAMBURG",DE,GB57METR12345678901234,XXX,HBUKGB4BXXX,HSBC BANK PLC,GB,1,0,0,1,0,0
1,2022-03-21,MT103 0001,PAYMENT FOR GOODS,USD,10000.0,CRED,ABC INDUSTRIES,ABC,None,INDUSTRIES,"NEW YORK, NY 10001 123 MAIN STREET",US,US12345678901234567890,XXX,ABCBUS33XXX,ABC BANK,US,ICBKCNBJGZU,CITIUS33,XYZ SUPPLIERS,XYZ,None,SUPPLIERS,123 HUANGPU ROAD,CN,CN123456789012345678,XXX,CITIUS33XXX,CITIBANK NA,US,1,0,0,0,2,0
2,2022-03-21,MT103 0001,INV NO. 12345 REF. 98765 SUPPLY OF GOODS AS PE...,USD,9899.0,CRED,ABC SUPPLIERS BV,ABC,SUPPLIERS,BV,None,NL,NL20ABNA0404875234,XXX,ABNANL2AXXX,None,NL,SCBLGB2LXXX,BNYMUS33XXX,AFRICAN EXPORT-IMPORT BANK,AFRICAN,EXPORT-IMPORT,BANK,None,NG,None,XXX,BNYMUS33XXX,BNY MELLON,US,0,1,0,1,1,1


In [1491]:
# Making sure that the columns are integer

df['rounding'] = df['rounding'].astype(int)
df['risky'] = df['risky'].astype(int)
df['smurf'] = df['smurf'].astype(int)
df['FATF'] = df['FATF'].astype(int)
df['shell'] = df['shell'].astype(int)
df['TBML'] = df['TBML'].astype(int)

df.dtypes

transaction_date                       datetime64[ns]
transaction_id                                 object
transaction_message                            object
transaction_currency                           object
transaction_amount                            float64
transaction_type                               object
originator_full_name                           object
originator_first_name                          object
originator_middle_names_patronymic             object
originator_last_name                           object
originator_address                             object
originator_country                             object
originator_account_number                      object
originator_branch_id                           object
originator_bic                                 object
originator_fi_name                             object
originator_fi_country                          object
incoming_intermediary_fi_bic                   object
outgoing_intermediary_fi_bic

In [1492]:
# Prompt used for ChatGPT-4: "Write a python script that counts the total sum per row in the dataframe of columns: 
# rounding; risky; smurf; FATF; shell; TBML, and add the total count to a new column called 'RISK'"

# List of columns to sum
columns_to_sum = ['rounding', 'risky', 'smurf', 'FATF', 'shell', 'TBML']

# Calculate the sum for each row and assign it to the new column 'RISK'
df['RISK'] = df[columns_to_sum].sum(axis=1)

df

,transaction_date,transaction_id,transaction_message,transaction_currency,transaction_amount,transaction_type,originator_full_name,originator_first_name,originator_middle_names_patronymic,originator_last_name,originator_address,originator_country,originator_account_number,originator_branch_id,originator_bic,originator_fi_name,originator_fi_country,incoming_intermediary_fi_bic,outgoing_intermediary_fi_bic,beneficiary_full_name,beneficiary_first_name,beneficiary_middle_names_patronymic,beneficiary_last_name,beneficiary_address,beneficiary_country,beneficiary_account_number,beneficiary_branch_id,beneficiary_bic,beneficiary_fi_name,beneficiary_fi_country,rounding,risky,smurf,FATF,shell,TBML,RISK
0,2022-03-21,MT103 0001,THIS IS A PAYMENT FOR TUNA SUPPLY,USD,5000.0,CRED,COMMERZBANK AG,COMMERZBANK,None,AG,None,DE,DE98765432101234567890,XXX,COBADEHHXXX,COMMERZBANK AG,DE,None,HBUKGB4BXXX,NORDFISCH GMBH,NORDFISCH,None,GMBH,"BODENSEE STR. 226, 22761 HAMBURG",DE,GB57METR12345678901234,XXX,HBUKGB4BXXX,HSBC BANK PLC,GB,1,0,0,1,0,0,2
1,2022-03-21,MT103 0001,PAYMENT FOR GOODS,USD,10000.0,CRED,ABC INDUSTRIES,ABC,None,INDUSTRIES,"NEW YORK, NY 10001 123 MAIN STREET",US,US12345678901234567890,XXX,ABCBUS33XXX,ABC BANK,US,ICBKCNBJGZU,CITIUS33,XYZ SUPPLIERS,XYZ,None,SUPPLIERS,123 HUANGPU ROAD,CN,CN123456789012345678,XXX,CITIUS33XXX,CITIBANK NA,US,1,0,0,0,2,0,3
2,2022-03-21,MT103 0001,INV NO. 12345 REF. 98765 SUPPLY OF GOODS AS PE...,USD,9899.0,CRED,ABC SUPPLIERS BV,ABC,SUPPLIERS,BV,None,NL,NL20ABNA0404875234,XXX,ABNANL2AXXX,None,NL,SCBLGB2LXXX,BNYMUS33XXX,AFRICAN EXPORT-IMPORT BANK,AFRICAN,EXPORT-IMPORT,BANK,None,NG,None,XXX,BNYMUS33XXX,BNY MELLON,US,0,1,0,1,1,1,4


In [1493]:
# Prompt used for ChatGPT-4: "Write a code that will show the entire row if column 'RISK' is equal or more than 3"

filtered_rows = df[df['RISK'] >= 3]
print(filtered_rows)

  transaction_date transaction_id  \
1       2022-03-21     MT103 0001   
2       2022-03-21     MT103 0001   

                                 transaction_message transaction_currency  \
1                                  PAYMENT FOR GOODS                  USD   
2  INV NO. 12345 REF. 98765 SUPPLY OF GOODS AS PE...                  USD   

   transaction_amount transaction_type originator_full_name  \
1             10000.0             CRED       ABC INDUSTRIES   
2              9899.0             CRED     ABC SUPPLIERS BV   

  originator_first_name originator_middle_names_patronymic  \
1                   ABC                               None   
2                   ABC                          SUPPLIERS   

  originator_last_name                  originator_address originator_country  \
1           INDUSTRIES  NEW YORK, NY 10001 123 MAIN STREET                 US   
2                   BV                                None                 NL   

  originator_account_number origina